# **Final Project - Data Analytics for Finance**
### 2023 - S2


<br>

Student Name | Student Number
--- | ---
Miguel Teodoro | 53127

<br>

**Signal:** Idiosyncratic skewness from the Fama-French 3-factor model

<br>


In [1]:
// Import the necessary packages/libraries

#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
#r "nuget:FSharp.Data,5.*"
#r "nuget: FSharp.Stats"
#r "nuget: DiffSharp-lite"
#r "nuget: NovaSBE.Finance, 0.2.0-beta1"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: Plotly.NET, 3.*"
#r "nuget: Plotly.NET.Interactive, 3.*"
#r "nuget: Quotes.YahooFinance, 0.0.5"

open Xunit
open FsUnit.Xunit
open FsUnitTyped
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats
open MathNet.Numerics.Statistics
open NovaSBE.Finance.Ols
open Quotes.YahooFinance
open DiffSharp

Installed Packages DiffSharp-lite, 1.0.7 FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 FsUnit.Xunit, 5.3.0 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2 Quotes.YahooFinance, 0.0.5 xunit, 2.4.2

Loading extensions from `C:\Users\migue\.nuget\packages\plotly.net.interactive\3.0.2\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Import files.

In [2]:
open NovaSBE.Finance.Portfolio

let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

let [<Literal>] IdAndReturnsFilePath = "data/id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "data/iskew_ff3_21d.csv"
let strategyName = "iskew21"


In [3]:
let idAndReturnsCsv = 
    CsvProvider<IdAndReturnsFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList
    
let mySignalCsv = 
    CsvProvider<MySignalFilePath,ResolutionFolder = __SOURCE_DIRECTORY__>.GetSample().Rows 
    |> Seq.toList

In [4]:
// store signals
let mySignals =
    mySignalCsv
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Signal = signal }
            Some signalRecord)

In [5]:
// store returns
let myReturns =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.Ret with
        | None -> None
        | Some ret ->
            let ret: SecurityReturn =
                { SecurityId = Other row.Id
                  Date = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Return= ret }
            Some ret)

In [6]:
// store market capitalizations
let myMktCaps =
    idAndReturnsCsv
    |> List.choose (fun row -> 
        match row.MarketEquity with
        | None -> None
        | Some mktCap ->
            let mktCap: WeightVariable =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Value = mktCap }
            Some mktCap)

### **3.2.** Strategy Analysis

In [7]:
// create a value-weighted backtest for the signal
let backtest = Backtest(returns=myReturns, signals=mySignals, nPortfolios=3, name = strategyName)
let vw = backtest.strategyValueWeighted(myMktCaps)

In [8]:
open NovaSBE.Finance

// get rf data
let ff3Lookup = 
    French.getFF3 French.Frequency.Monthly
    |> Array.map (fun x -> DateTime(x.Date.Year, x.Date.Month, 1), x)
    |> Map

// apply it to calculate excess returns
let vwExcessReturns =
    vw.Returns
    |> List.map(fun x -> 
        { Name = x.Name
          Index = x.Index
          Month = x.Month
          Return = x.Return - ff3Lookup[x.Month].Rf})

In [9]:
// get the long portfolio
let long =
    vwExcessReturns
    |> List.filter (fun row -> row.Index = 3)
    
// get the short portfolio
let short =
    vwExcessReturns
    |> List.filter (fun row -> row.Index = 1)

// get the second tercile portfolio (won't be used)
// let medium =
//     vwExcessReturns
//     |> List.filter (fun row -> row.Index = 2)

// create long-short portfolio
let longShort =
    vw.Returns
    |> List.groupBy (fun x -> x.Month)
    |> List.map (fun (month, xs) ->
        let long = xs |> List.find (fun x -> x.Index = 3)
        let short = xs |> List.find (fun x -> x.Index = 1)
        { Name = "Long-Short"
          Index = 0
          Month = long.Month
          Return = long.Return - short.Return})

// get the value-weighted stock market excess returns portfolio (fama-french data comes as value-weighted)
let market =
    // impose limit dates to compare with portfolios
    let maxDate = vwExcessReturns |> List.map(fun x -> x.Month) |> List.max
    let minDate = vwExcessReturns |> List.map(fun x -> x.Month) |> List.min
    French.getFF3 French.Frequency.Monthly
    |> Array.toList
    |> List.filter(fun x -> x.Date <= maxDate)
    |> List.filter(fun x -> x.Date >= minDate)
    |> List.map(fun x ->
        { Name = "Market"
          Index = 10
          Month = x.Date
          Return = x.MktRf })

In [10]:
// function to calculate cumulative returns
let cumulativeReturn (xs: PortfolioReturn list) =
    let mutable cr = 1.0
    let sorted = xs |> List.sortBy(fun x -> x.Month)
    [ for x in sorted do 
        cr <- cr * (1.0 + x.Return)
        { x with Return = cr } ]

In [11]:
// function to plot cumulative returns
let plotCumulativeRets (xs: PortfolioReturn list) =
    xs
    |> cumulativeReturn
    |> List.map(fun x -> x.Month, x.Return)
    |> Chart.Line 
    |> Chart.withLegendStyle(Orientation=StyleParam.Orientation.Horizontal)

// creates chart and shows
let combinedChart =
    [plotCumulativeRets long |> Chart.withTraceInfo(Name="Long Portfolio");
     plotCumulativeRets short |> Chart.withTraceInfo(Name="Short Portfolio");
     plotCumulativeRets market |> Chart.withTraceInfo(Name="Market Excess Returns");
     plotCumulativeRets longShort |> Chart.withTraceInfo(Name="Long-Short Portfolio");]
    |> Chart.combine
    |> Chart.withTitle "Cumulative Returns for Value-Weighted Portfolios"

combinedChart

<!-- Plotly chart will be drawn inside this DIV -->

In [12]:
// function calculate annual standard deviations from monthly
let annualizeMonthlyStdDev monthlyStdDev = sqrt(12.0) * monthlyStdDev

// function to annualize portfolios standard deviations
let stdDevPortfolioAnnualized (xs: PortfolioReturn list) =
    xs
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

// function to leverage portfolio to have 10% volatility
let vol10 (xs: PortfolioReturn list) =
    xs
    |> List.map(fun x ->
        let w = 0.1 / stdDevPortfolioAnnualized xs
        { Name = x.Name
          Index = x.Index
          Month = x.Month
          Return = x.Return * w
        })

// plots leveraged portfolios
let combinedChartVol =
    [plotCumulativeRets (vol10 long) |> Chart.withTraceInfo(Name="Long Portfolio");
     plotCumulativeRets (vol10 short) |> Chart.withTraceInfo(Name="Short Portfolio");
     plotCumulativeRets (vol10 market) |> Chart.withTraceInfo(Name="Market Excess Returns");
     plotCumulativeRets (vol10 longShort) |> Chart.withTraceInfo(Name="Long-Short Portfolio");]
    |> Chart.combine
    |> Chart.withTitle "Cumulative Returns for Value-Weighted Portfolios with Constant Leverage"

combinedChartVol

<!-- Plotly chart will be drawn inside this DIV -->

In [13]:
// splits the data in two halves
let splitInTwo (xs: PortfolioReturn list) =
    xs
    |> List.sortBy (fun x -> x.Month)
    |> List.splitInto 2


In [14]:
// applies the splitting function and stores the halves

let long1 = (splitInTwo long)[0]
let long2 = (splitInTwo long)[1]

let longShort1 = (splitInTwo longShort)[0]
let longShort2 = (splitInTwo longShort)[1]

let market1 = (splitInTwo market)[0]
let market2 = (splitInTwo market)[1]

In [15]:
// function to calculate annualized returns from monthly returns
let annualizeMonthlyReturns monthlyReturns = 12.0 * monthlyReturns

// function to annualize portfolios average returns
let returnsPortfolioAnnualized (xs: PortfolioReturn list) =
    xs
    |> Seq.averageBy (fun x -> x.Return)
    |> annualizeMonthlyReturns


In [16]:
// for each portfolio stores annualized returns

let returnLong = returnsPortfolioAnnualized long
let returnLong1 = returnsPortfolioAnnualized long1
let returnLong2 = returnsPortfolioAnnualized long2

let returnLongShort = returnsPortfolioAnnualized longShort
let returnLongShort1 = returnsPortfolioAnnualized longShort1
let returnLongShort2 = returnsPortfolioAnnualized longShort2

let returnMarket = returnsPortfolioAnnualized market
let returnMarket1 = returnsPortfolioAnnualized market1
let returnMarket2 = returnsPortfolioAnnualized market2

In [17]:
// for each portfolio stores annualized volatility

let stdDevLong = stdDevPortfolioAnnualized long
let stdDevLong1 = stdDevPortfolioAnnualized long1
let stdDevLong2 = stdDevPortfolioAnnualized long2

let stdDevLongShort = stdDevPortfolioAnnualized longShort
let stdDevLongShort1 = stdDevPortfolioAnnualized longShort1
let stdDevLongShort2 = stdDevPortfolioAnnualized longShort2

let stdDevMarket = stdDevPortfolioAnnualized market
let stdDevMarket1 = stdDevPortfolioAnnualized market1
let stdDevMarket2 = stdDevPortfolioAnnualized market2

Calculate Sharpe

In [18]:
// function to calculate portfolios sharpe ratio
let srPortfolio (xs: PortfolioReturn list) = 
    (returnsPortfolioAnnualized xs) / (stdDevPortfolioAnnualized xs)

In [19]:
// for each portfolio stores sharpe ratio

let srLong = srPortfolio long
let srLong1 = srPortfolio long1
let srLong2 = srPortfolio long2

let srLongShort = srPortfolio longShort
let srLongShort1 = srPortfolio longShort1
let srLongShort2 = srPortfolio longShort2

let srMarket = srPortfolio market
let srMarket1 = srPortfolio market1
let srMarket2 = srPortfolio market2

CAPM and FF3 alphas

In [20]:
// creates regression data type
type RegData =
     {  Date: DateTime
        Portfolio: float
        MktRf: float
        Hml: float
        Smb: float }

// function to adapt portfolio returns to regression data type
let adaptRD (xs: PortfolioReturn list) =
    xs
    |> List.map (fun x ->
        let xff3 = ff3Lookup[x.Month]
        { Date = x.Month
          Portfolio = x.Return
          MktRf = xff3.MktRf
          Hml = xff3.Hml
          Smb = xff3.Smb })

In [22]:
// functions to fit CAPM and Fama-French-3 factor models

let fitCAPM (xs: RegData list) =
    Ols("Portfolio ~ MktRf", xs).fit()

let fitFF3 (xs: RegData list) =
    Ols("Portfolio ~ MktRf + Hml + Smb", xs).fit()

In [23]:
// creates a type to suit regression outputs
type outputReg =
    { Alpha_CAPM: float
      Alpha_TstatCAPM: float
      MktRf_CoefCAPM: float
      MktRf_TstatCAPM: float
      R2_CAPM: float
      IR_CAPM: float

      Alpha_FF3: float
      Alpha_TstatFF3: float
      MktRf_CoefFF3: float
      MktRf_TstatFF3: float
      Hml_CoefFF3: float
      Hml_TstatFF3: float
      Smb_CoefFF3: float
      Smb_TstatFF3: float
      R2_FF3: float
      IR_FF3: float
      }

In [24]:
// function that fits CAPM and FF3 regressions to a portfolio and stores the models outputs
let fitRegs(xs: RegData list) =

    let capm = fitCAPM(xs)
    let ff3 = fitFF3(xs)

    // calculate IR
    let capmStDevResiduals = (Seq.stDev capm.resid)
    let capmIR = capm.coefs["Intercept"] / capmStDevResiduals

    let ff3StDevResiduals = (Seq.stDev ff3.resid)
    let ff3IR = ff3.coefs["Intercept"] / ff3StDevResiduals

    { Alpha_CAPM = capm.coefs["Intercept"]
      Alpha_TstatCAPM = capm.tvalues["Intercept"]
      MktRf_CoefCAPM = capm.coefs["MktRf"]
      MktRf_TstatCAPM = capm.tvalues["MktRf"]
      R2_CAPM = capm.rsquared
      IR_CAPM = capmIR

      Alpha_FF3 = ff3.coefs["Intercept"]
      Alpha_TstatFF3 = ff3.tvalues["Intercept"]
      MktRf_CoefFF3 = ff3.coefs["MktRf"]
      MktRf_TstatFF3 = ff3.tvalues["MktRf"]
      Hml_CoefFF3 = ff3.coefs["Hml"]
      Hml_TstatFF3 = ff3.tvalues["Hml"]
      Smb_CoefFF3 = ff3.coefs["Smb"]
      Smb_TstatFF3 = ff3.tvalues["Smb"]
      R2_FF3 = ff3.rsquared
      IR_FF3 = ff3IR
    }

In [25]:
// regressions of both the Long-Only and the Long-Short portfolios

let regLong = fitRegs (adaptRD long)
let regLong1 = fitRegs (adaptRD long1)
let regLong2 = fitRegs (adaptRD long2)

let regLongShort = fitRegs (adaptRD longShort)
let regLongShort1 = fitRegs (adaptRD longShort1)
let regLongShort2 = fitRegs (adaptRD longShort2)

In [26]:
// eases visibility in following table
let roundDisplay (value: float) = Math.Round(value, 3).ToDisplayString()

In [27]:
// creates table with portfolio statistics

let header = 
    [ 
    "<b>Portfolio"      ; "<b>Avg. Excess Return"       ; "<b>Avg. Volatility"          ; "<b>Sharpe Ratio"         ; "<b>CAPM Alpha"                          ; "<b>CAPM Alpha t-stat"                        ; "<b>CAPM R-squared"                   ; "<b>CAPM IR"                          ; "<b>FF3 Alpha"                        ; "<b>FF3 Alpha t-stat"                     ; "<b>FF3 R-squared"                ; "<b>FF3 IR"                       
    ]

let rows = 
    [
     ["Long-only"       ; roundDisplay returnLong       ; roundDisplay stdDevLong       ; roundDisplay srLong       ; roundDisplay regLong.Alpha_CAPM          ; roundDisplay regLong.Alpha_TstatCAPM          ; roundDisplay regLong.R2_CAPM          ; roundDisplay regLong.IR_CAPM          ; roundDisplay regLong.Alpha_FF3        ; roundDisplay regLong.Alpha_TstatFF3       ; roundDisplay regLong.R2_FF3       ; roundDisplay regLong.IR_FF3       ]           
     ["Long-only 1st"   ; roundDisplay returnLong1      ; roundDisplay stdDevLong1      ; roundDisplay srLong1      ; roundDisplay regLong1.Alpha_CAPM         ; roundDisplay regLong1.Alpha_TstatCAPM         ; roundDisplay regLong1.R2_CAPM         ; roundDisplay regLong1.IR_CAPM         ; roundDisplay regLong1.Alpha_FF3       ; roundDisplay regLong1.Alpha_TstatFF3      ; roundDisplay regLong1.R2_FF3      ; roundDisplay regLong1.IR_FF3      ]   
     ["Long-only 2nd"   ; roundDisplay returnLong2      ; roundDisplay stdDevLong2      ; roundDisplay srLong2      ; roundDisplay regLong2.Alpha_CAPM         ; roundDisplay regLong2.Alpha_TstatCAPM         ; roundDisplay regLong2.R2_CAPM         ; roundDisplay regLong2.IR_CAPM         ; roundDisplay regLong2.Alpha_FF3       ; roundDisplay regLong2.Alpha_TstatFF3      ; roundDisplay regLong2.R2_FF3      ; roundDisplay regLong2.IR_FF3      ]
     ["Long-Short"      ; roundDisplay returnLongShort  ; roundDisplay stdDevLongShort  ; roundDisplay srLongShort  ; roundDisplay regLongShort.Alpha_CAPM     ; roundDisplay regLongShort.Alpha_TstatCAPM     ; roundDisplay regLongShort.R2_CAPM     ; roundDisplay regLongShort.IR_CAPM     ; roundDisplay regLongShort.Alpha_FF3   ; roundDisplay regLongShort.Alpha_TstatFF3  ; roundDisplay regLongShort.R2_FF3  ; roundDisplay regLongShort.IR_FF3  ] 
     ["Long-Short 1st"  ; roundDisplay returnLongShort1 ; roundDisplay stdDevLongShort1 ; roundDisplay srLongShort1 ; roundDisplay regLongShort1.Alpha_CAPM    ; roundDisplay regLongShort1.Alpha_TstatCAPM    ; roundDisplay regLongShort1.R2_CAPM    ; roundDisplay regLongShort1.IR_CAPM    ; roundDisplay regLongShort1.Alpha_FF3  ; roundDisplay regLongShort1.Alpha_TstatFF3 ; roundDisplay regLongShort1.R2_FF3 ; roundDisplay regLongShort1.IR_FF3 ] 
     ["Long-Short 2nd"  ; roundDisplay returnLongShort2 ; roundDisplay stdDevLongShort2 ; roundDisplay srLongShort2 ; roundDisplay regLongShort2.Alpha_CAPM    ; roundDisplay regLongShort2.Alpha_TstatCAPM    ; roundDisplay regLongShort2.R2_CAPM    ; roundDisplay regLongShort2.IR_CAPM    ; roundDisplay regLongShort2.Alpha_FF3  ; roundDisplay regLongShort2.Alpha_TstatFF3 ; roundDisplay regLongShort2.R2_FF3 ; roundDisplay regLongShort2.IR_FF3 ]
     ["Market"          ; roundDisplay returnMarket     ; roundDisplay stdDevMarket     ; roundDisplay srMarket     ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
     ["Market 1st"      ; roundDisplay returnMarket1    ; roundDisplay stdDevMarket1    ; roundDisplay srMarket1    ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
     ["Market 2nd"      ; roundDisplay returnMarket2    ; roundDisplay stdDevMarket2    ; roundDisplay srMarket2    ; ""                                       ; ""                                            ; ""                                    ; ""                                    ; ""                                    ; ""                                        ; ""                                ; ""                                ] 
    ]
 

Chart.Table(header, rows) 
|> Chart.withSize (1550, 450) 
|> Chart.withTitle("Performance Measures")

<!-- Plotly chart will be drawn inside this DIV -->

## **3.3.** Strategy as part of a diversified portfolio

In [28]:
// creates type for stock data
type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

// tickers list, in this case will only use Bond and Equity indexes to construct the 60/40 portfolio
let tickers = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

// get tickers data
let tickPrices = 
    YahooFinance.History(
        tickers,
        startDate = DateTime(2000,1,1),
        interval = Interval.Monthly)

// calculate returns from prices
let pricesToReturns (symbol, adjPrices: list<Quote>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })

// group returns by symbol
let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

In [29]:
// get ff3 data in list format
let ff3 = French.getFF3 French.Frequency.Monthly |> Array.toList

// store rf rates
let rf = Map [ for x in ff3 do x.Date, x.Rf ]

// calculate excess returns for each of the downloaded tickers
let standardInvestmentsExcess =
    let maxff3Date = ff3 |> List.map(fun x -> x.Date) |> List.max
    let minff3Date = ff3 |> List.map(fun x -> x.Date) |> List.min
    tickReturns
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.filter(fun x -> x.Date >= minff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"no rf for {x.Date}"
        | Some rf -> { x with Return = x.Return - rf })

In [30]:
// adjust long-only to StockData type
let longAdjusted =
  long
  |> List.map(fun x ->
                { Symbol = "Long-Only"
                  Date = x.Month
                  Return = x.Return
                })

// adjust long-short to StockData type
let longShortAdjusted =
  longShort
  |> List.map(fun x ->
                { Symbol = "Long-Short"
                  Date = x.Month
                  Return = x.Return
                })

In [31]:
// join all data for Long portfolio
let allStockDataLong =
    List.concat [|standardInvestmentsExcess; longAdjusted|]

// join all data for Long-Short portfolio
let allStockDataLongShort =
    List.concat [|standardInvestmentsExcess; longShortAdjusted|]

In [32]:
// necessary for dsharp
Formatter.Register<Tensor>(
    Action<_, _>(fun t (writer: IO.TextWriter) -> fprintfn writer "%120A" t),"text/plain")
Formatter.Register<Tensor>(
    Action<_, _>(fun t (writer: IO.TextWriter) -> fprintfn writer "%120A" t),"text/html")
Formatter.SetPreferredMimeTypesFor(typeof<Tensor>, "text/plain")

In [33]:
// store tickers for Long-Only comparison
let tickersLong = ["Long-Only"; "VTI"; "BND"]

// store tickers for Long-Short comparison
let tickersLongShort = ["Long-Short"; "VTI"; "BND"]


In [34]:
// function to calculate covariance between returns
let getCov xId yId (data: StockData list) =
    let xRet = 
        data
        |> List.filter(fun x -> x.Symbol = xId)
        |> List.map(fun x -> x.Date, x.Return)
        |> Map
    let yRet = 
        data
        |> List.filter(fun x -> x.Symbol = yId)
        |> List.map(fun x -> x.Date, x.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs

// function to create covariances matrix
let covariances (ticks: string list) (data: StockData list) =
    [ for rowTick in ticks do 
        [ for colTick in ticks do
            getCov rowTick colTick data]]
    |> dsharp.tensor

// get covariances for both portfolios
let covLong = covariances tickersLong allStockDataLong
let covLongShort = covariances tickersLongShort allStockDataLongShort


In [35]:
// function to calculate mean returns by ticker
let meansByTick (data: StockData list) =
    data
    |> List.groupBy (fun x -> x.Symbol)
    |> List.map (fun (sym, xs) ->
        let symAvg = xs |> List.averageBy (fun x -> x.Return)
        sym, symAvg)
    |> Map

// get means by ticker to dsharp format
let means (tickers: string list) (data: StockData list) =
    // Make sure ticker avg returns are ordered by our ticker list
    [ for ticker in tickers do meansByTick(data)[ticker]]
    |> dsharp.tensor

// get means for both portfolios
let meansLong = means tickersLong allStockDataLong
let meansLongShort = means tickersLongShort allStockDataLongShort

In [36]:
// function to calculate optimal weights according to mve
let calcWeights (covars: Tensor) (means: Tensor) (tickers: string list) =
    let w' = dsharp.solve(covars,means)
    let w = w' / w'.sum()
    let weights =
        Seq.zip tickers (w.toArray1D<float>())
        |> Map.ofSeq
    weights  

// mean-variance optimal weights for the Long-Only portfolio
let weightsLong = calcWeights covLong meansLong tickersLong

// mean-variance optimal weights for the Long-Short portfolio
let weightsLongShort = calcWeights covLongShort meansLongShort tickersLongShort

// assign optimal weights for 60/40 portfolio
let weights6040 = Map [("VTI",0.6);("BND",0.4)]

In [37]:
// function to transform data in order to get complete data
let getCompleteData (tickers: string list) (data: StockData list) = 

    // sort data by date
    let stockDataByDate (data: StockData list) =
        data
        |> List.groupBy(fun x -> x.Date) // group all symbols on the same date together.
        |> List.sortBy (fun (dt, xs) -> dt) 

    // find the first array element where there are as many stocks as we have symbols
    let allAssetsStart (tickers: string list) (data: StockData list) =
        stockDataByDate data
        |> List.find(fun (month, stocks) -> stocks.Length = tickers.Length)
        |> fst

    // find the last array element where there are as many stocks as we have symbols
    let allAssetsEnd (tickers: string list) (data: StockData list) =
        stockDataByDate data
        |> List.findBack(fun (month, stocks) -> stocks.Length = tickers.Length)
        |> fst

    // get the data where all tickers are available
    let stockDataByDateComplete (tickers: string list) (data: StockData list) =
        stockDataByDate data
        |> List.filter(fun (date, stocks) -> 
            date >= allAssetsStart tickers data &&
            date <= allAssetsEnd tickers data)

    // check if we got all the data
    let checkOfCompleteData (tickers: string list) (data: StockData list) =
        stockDataByDateComplete tickers data
        |> List.map snd
        |> List.filter(fun x -> x.Length <> tickers.Length) // discard rows where we have all symbols.

    if not (List.isEmpty (checkOfCompleteData tickers data)) then 
            failwith "stockDataByDateComplete has months with missing stocks"
    
    // return final list
    stockDataByDateComplete tickers data

In [38]:
// get complete data for Long-Only portfolio
let completeDataLong = getCompleteData tickersLong allStockDataLong

// get complete data for Long-Short portfolio
let completeDataLongShort = getCompleteData tickersLongShort allStockDataLongShort

In [39]:
// function to calculate monthly returns of portfolio according to given weighting scheme
let portfolioMonthReturn weights monthData =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

In [40]:
// form list with Long-Only MVE portfolio returns
let portMveLong =
    completeDataLong
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE-Long"
          Date = date
          Return = portfolioMonthReturn weightsLong data })

// form list with Long-Short MVE portfolio returns
let portMveLongShort =
    completeDataLongShort
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE-Long-Short"
          Date = date
          Return = portfolioMonthReturn weightsLongShort data })

// form list with 60/40 portfolio returns
let port6040 = 
    completeDataLong
    |> List.map(fun (date, data) -> 
        { Symbol = "60/40"
          Date = date 
          Return = portfolioMonthReturn weights6040 data} )

In [41]:
// function to calculate cumulative returns
let cumulateReturns (xs: StockData list) =
    let mutable cr = 1.0
    [ for x in xs do 
        cr <- cr * (1.0 + x.Return)
        { x with Return = cr } ]

// apply to the three portfolios
let portMveLongCumulative = portMveLong |> cumulateReturns
let portMveLongShortCumulative = portMveLongShort |> cumulateReturns
let port6040Cumulative = port6040 |> cumulateReturns

// function to create charts
let chartReturns (cumulativeReturns: StockData list) = 
    cumulativeReturns
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line

// create charts for each portfolio
let chartLong = chartReturns portMveLongCumulative |> Chart.withTraceInfo(Name="MVE Long-Only")
let chartLongShort = chartReturns portMveLongShortCumulative |> Chart.withTraceInfo(Name="MVE Long-Short")
let chart6040 = chartReturns port6040Cumulative |> Chart.withTraceInfo(Name="60/40")

// combine charts to single plot
let chartCombined =
    [ chartLong; chartLongShort; chart6040 ]
    |> Chart.combine

chartCombined

<!-- Plotly chart will be drawn inside this DIV -->

In [42]:
// adapt standard deviation and leverage to 10% volatility functions to the stock data type

let stdDevPortfolioAnnualized (xs: StockData list) =
    xs
    |> Seq.stDevBy (fun x -> x.Return)
    |> annualizeMonthlyStdDev

let vol10 (xs: StockData list) =
    xs
    |> List.map(fun x ->
        let w = 0.1 / stdDevPortfolioAnnualized xs
        { Symbol = x.Symbol
          Date = x.Date
          Return = x.Return * w
        })

// apply to the three portfolios
let portMveLongVol = portMveLong |> vol10 |> cumulateReturns
let portMveLongShortVol = portMveLongShort |> vol10 |> cumulateReturns
let port6040Vol = port6040 |> vol10 |> cumulateReturns

// create charts for each portfolio
let chartLongVol = chartReturns portMveLongVol |> Chart.withTraceInfo(Name="MVE Long-Only")
let chartLongShortVol = chartReturns portMveLongShortVol |> Chart.withTraceInfo(Name="MVE Long-Short")
let chart6040Vol = chartReturns port6040Vol |> Chart.withTraceInfo(Name="60/40")

// combine charts to single plot
let chartCombinedVol =
    [ chartLongVol; chartLongShortVol; chart6040Vol ]
    |> Chart.combine

chartCombinedVol

<!-- Plotly chart will be drawn inside this DIV -->

In [43]:
// optimal weighting scheme for the long-only portfolio by mve optimization
weightsLong

key,value
BND,0.8181894421577454
Long-Only,0.18841750919818878
VTI,-0.006606990937143564


In [44]:
// optimal weighting scheme for the long-short portfolio by mve optimization
weightsLongShort

key,value
BND,0.6708157062530518
Long-Short,0.15380489826202393
VTI,0.17537939548492432


In [97]:
// adapt average return and sharpe ratio functions to the stock data type

let returnsPortfolioAnnualized (xs: StockData list) =
    xs
    |> Seq.averageBy (fun x -> x.Return)
    |> annualizeMonthlyReturns

let srPortfolio (xs: StockData list) = 
    (returnsPortfolioAnnualized xs) / (stdDevPortfolioAnnualized xs)

In [100]:
// calculate average return, volatility and sharpe ratio for the long-only, long-short and 60/40 portfolios

let returnMveLong = returnsPortfolioAnnualized portMveLong
let returnMveLongShort = returnsPortfolioAnnualized portMveLongShort
let return6040 = returnsPortfolioAnnualized port6040

let stdDevMveLong = stdDevPortfolioAnnualized portMveLong
let stdDevMveLongShort = stdDevPortfolioAnnualized portMveLongShort
let stdDev6040 = stdDevPortfolioAnnualized port6040

let srMveLong = srPortfolio portMveLong
let srMveLongShort = srPortfolio portMveLongShort
let sr6040 = srPortfolio port6040

In [101]:
let header = 
    [ 
    "<b>Portfolio"      ; "<b>Avg. Excess Return"           ; "<b>Avg. Volatility"              ; "<b>Sharpe Ratio"                                
    ]

let rows = 
    [
     ["MVE Long-Only"   ; roundDisplay returnMveLong        ; roundDisplay stdDevMveLong        ; roundDisplay srMveLong            ]           
     ["MVE Long-Short"  ; roundDisplay returnMveLongShort   ; roundDisplay stdDevMveLongShort   ; roundDisplay srMveLongShort       ] 
     ["60/40"           ; roundDisplay return6040           ; roundDisplay stdDev6040           ; roundDisplay sr6040               ] 
    ]
 

Chart.Table(header, rows) 
|> Chart.withSize (1000, 450) 
|> Chart.withTitle("Performance Measures")

<!-- Plotly chart will be drawn inside this DIV -->